# 1 Import packages and define useful functions

In [2]:
import pandas as pd
import numpy as np
from scipy import signal
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [3]:
def get_shortest_array(a,b):
    '''
    input - a,b arrays (pandas.values) arrays
    output - returns length of shortest array using first index of np.shape
    '''
    
    len_a = np.shape(a)[0]
    len_b = np.shape(b)[0]
    
    return min(len_a,len_b)

In [4]:
def plot_cor(x, y, window, filename, x_name, y_name):
    '''
    x (df) - first time series
    y (df) - second time series
    window (int) - period length
    filename - output filename as .png
    x_name - first time series name (for title)
    y_name - second time series name (for title)
    Note that x and y don't have to be the same length. the 'get_shortest_array' function will be used to automatically trim the time series
    '''
    shortest_time_series_len = get_shortest_array(x,y)
    plt.plot(x.iloc[:shortest_time_series_len].rolling(window).corr(y.iloc[:shortest_time_series_len]).fillna(method='bfill'));
    
    # rotate x-axis labels so date labels all fit in properly
    plt.xticks(rotation=90);
    plt.title('20-day rolling correlation between '+str(x_name)+ ' and '+str(y_name))
    plt.savefig(str(filename),dpi=600)


In [5]:
def equity_d2m(dataframe):
    '''
    for a dataframe with daily OHLC equity data, convert this to monthly OHLC data.
    '''
    
    return pd.DataFrame({"Open":dataframe.Open.resample('MS').first(),
                         "High":dataframe.High.resample('MS').max(),
                         "Low":dataframe.Low.resample('MS').min(),
                         "Close":dataframe.Close.resample('MS').last()})

# 2 Import data: Hang Seng and macroeconomic indicators

In [20]:
# daily time series
HK50 = pd.read_csv('data/HS/HK50.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
DXY = pd.read_csv('data/macro/daily/DXY.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
HSIL = pd.read_csv('data/macro/daily/^HSIL.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
SPY = pd.read_csv('data/macro/daily/SPY.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')

# currencies
AUDUSD = pd.read_csv('data/macro/daily/AUD_USD.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
AUDCNY = pd.read_csv('data/macro/daily/AUD_CNY.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
CNYHKD = pd.read_csv('data/macro/daily/CNY_HKD.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
USDHKD = pd.read_csv('data/macro/daily/USD_HKD.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
HKDEUR = pd.read_csv('data/macro/daily/HKD_EUR.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
HKDAUD = pd.read_csv('data/macro/daily/HKD_AUD.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')

In [21]:
# monthly time series
US10Y = pd.read_csv('data/macro/monthly/US_10Y.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
CN10Y = pd.read_csv('data/macro/monthly/CN_10Y.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
CAIXIN = pd.read_csv('data/macro/monthly/Caixin_PMI.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')

# CPI
US_CPI_FRED_all = pd.read_csv('data/macro/monthly/CPIAUCSL.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
US_CPI_OECD = pd.read_csv('data/macro/monthly/US_CPI_OECD.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
CN_CPI_OECD = pd.read_csv('data/macro/monthly/CN_CPI_OECD.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')

# M3 money supply
US_M2 = pd.read_csv('data/macro/monthly/US_M2.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
CN_M2 = pd.read_csv('data/macro/monthly/CN_M2.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
EU_M2 = pd.read_csv('data/macro/monthly/EU_M2.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')

In [16]:
# quarterly time series
CN_GDP = pd.read_csv('data/macro/quarterly/CN_GDP.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')
US_GDP = pd.read_csv('data/macro/quarterly/US_GDP.csv', index_col=0, parse_dates=True, infer_datetime_format=True, thousands=',')

In [10]:
CN10Y

,China 10yr Benchmark Yield %,China CPI YoY %,China Real 10yr Rate (10yr Benchmark - CPI All Items)
Date,,,
2000-09-30,2.79,0.0,2.79
2000-10-31,2.75,0.0,2.75
2000-11-30,2.97,1.3,1.67
2000-12-31,3.01,1.5,1.51
2001-01-31,3.00,1.2,1.80
...,...,...,...
2022-07-31,2.76,2.7,0.06
2022-08-31,2.65,2.5,0.15
2022-09-30,2.78,2.8,-0.02
